# Data Wrangling
In data preparation includes:
* loading
* cleaning
* transforming
* rearranging

## Combining and merging data sets
There are different ways to combine the dataset together in pandas:
* [pandas.merge](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.merge.html#pandas.merge) -- a SQL or relational database _join_ method
* [pandas.concat](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html) -- glues or stacks together objects along an axis
* combine_first -- splicing together overlapping data to fill in missing values in one object with values from another.

 ### Database-Style DataFrame join
Pandas provides a single funciton, __merge__, as the entry point for all standard database join operations between DataFrame objects. The function is also available as a DataFrame instance method.

There are several cases in Database __join__:
* __one-to-one__ joins: joining two DataFrame objects on their indexes which must contain unique values
* __many-to-one__ joins: joining a unique index to one or more columns in a DataFrame
* __many-to-many__ joins: joining columns on columns. If a key combination appears more than once in both tables, the resulting table will have the __Cartesian product__ of the associated data. 

In [1]:
import pandas as pd

In [2]:
left = pd.DataFrame({'key1' : ['K0', 'K0', 'K1', 'K2'],
                    'key2' : ['K0', 'K1', 'K0', 'K1'],
                    'A' : [1, 2, 3, 4],
                    'B': [0.1, 0.2, 0.3,0.4]})
left

,A,B,key1,key2
0,1,0.1,K0,K0
1,2,0.2,K0,K1
2,3,0.3,K1,K0
3,4,0.4,K2,K1


In [3]:
right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'k2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': [-0.1, -0.2, -0.3, -0.4],
                      'D': [-1, -2, -3, -4]})
right

,C,D,key1,key2
0,-0.1,-1,K0,K0
1,-0.2,-2,K1,K0
2,-0.3,-3,K1,K0
3,-0.4,-4,k2,K0


In [4]:
result = pd.merge(left, right, on = ['key1', 'key2'])
result

,A,B,key1,key2,C,D
0,1,0.1,K0,K0,-0.1,-1
1,3,0.3,K1,K0,-0.2,-2
2,3,0.3,K1,K0,-0.3,-3


In __merge__ function, there is a __how__ argument to specify how to determine which keys are to be included in the resulting table. In general, if a key combination __does not appear__ in either the left or right tables, the values in the joined table will be __NA__. By default, __merge__ function use _inner join_. Here is a summary:

__Merge method | SQL Join Name | Description 
---------------|----------------------------
left | LEFT OUTER JOIN | Use keys from left frame only
right| RIGHT OUTER JOIN |Use keys from right frame only
outer| FULL OUTER JOIN | Use union of keys from both frames
inner| INNER JOIN | Use intersection of keys from both frames

In [5]:
result = pd.merge(left, right, how = 'left', on = ['key1', 'key2'])
result

,A,B,key1,key2,C,D
0,1,0.1,K0,K0,-0.1,-1.0
1,2,0.2,K0,K1,NaN,NaN
2,3,0.3,K1,K0,-0.2,-2.0
3,3,0.3,K1,K0,-0.3,-3.0
4,4,0.4,K2,K1,NaN,NaN


In [6]:
result = pd.merge(left, right, how = 'right', on = ['key1', 'key2'])
result

,A,B,key1,key2,C,D
0,1.0,0.1,K0,K0,-0.1,-1
1,3.0,0.3,K1,K0,-0.2,-2
2,3.0,0.3,K1,K0,-0.3,-3
3,NaN,NaN,k2,K0,-0.4,-4


In [7]:
result = pd.merge(left, right, how = 'outer', on = ['key1', 'key2'])
result

,A,B,key1,key2,C,D
0,1.0,0.1,K0,K0,-0.1,-1.0
1,2.0,0.2,K0,K1,NaN,NaN
2,3.0,0.3,K1,K0,-0.2,-2.0
3,3.0,0.3,K1,K0,-0.3,-3.0
4,4.0,0.4,K2,K1,NaN,NaN
5,NaN,NaN,k2,K0,-0.4,-4.0


The __indicator__ argument will shows which values are taken:

__Observation Origin__ | __\_merge value__
---------------------|----------------
Merge key only in 'left' frame|left_only
Merge key only in 'right' frame|right_only
Merge key in bot frames|both

In [8]:
result = pd.merge(left, right, how = 'left', on = ['key1', 'key2'], indicator = True)
result

,A,B,key1,key2,C,D,_merge
0,1,0.1,K0,K0,-0.1,-1.0,both
1,2,0.2,K0,K1,NaN,NaN,left_only
2,3,0.3,K1,K0,-0.2,-2.0,both
3,3,0.3,K1,K0,-0.3,-3.0,both
4,4,0.4,K2,K1,NaN,NaN,left_only


In [9]:
result = pd.merge(left, right, how = 'left', on = ['key1', 'key2'], indicator = 'take_on_columns')
result

,A,B,key1,key2,C,D,take_on_columns
0,1,0.1,K0,K0,-0.1,-1.0,both
1,2,0.2,K0,K1,NaN,NaN,left_only
2,3,0.3,K1,K0,-0.2,-2.0,both
3,3,0.3,K1,K0,-0.3,-3.0,both
4,4,0.4,K2,K1,NaN,NaN,left_only


Sometimes two DataFrame may share the same column names, __suffix__ can disambiguate the result columns:

In [10]:
left = pd.DataFrame({'k' : ['k0', 'k1', 'k2'], 'v': [1, 2, 3]})

right = pd.DataFrame({'k' : ['k0', 'k0', 'k3'], 'v': [4, 5, 6]})

left

,k,v
0,k0,1
1,k1,2
2,k2,3


In [11]:
right

,k,v
0,k0,4
1,k0,5
2,k3,6


In [12]:
result = pd.merge(left, right, on = 'k', suffixes = ['_l', '_r'])
result

,k,v_l,v_r
0,k0,1,4
1,k0,1,5


In some cases, the merge key or keys in a DataFrame will be found in its index. 

In [13]:
left = pd.DataFrame( {'a': ['a10', 'a11', 'a12'],
                      'b': ['b10', 'b11', 'b12']},
                      index = ['k0', 'k1', 'k2'])
left

,a,b
k0,a10,b10
k1,a11,b11
k2,a12,b12


In [14]:
right = pd.DataFrame( {'c' : ['c10', 'c20', 'c30'],
                       'd' : ['d10', 'd20', 'd30']},
                       index = ['k0', 'k2', 'k3'])
right

,c,d
k0,c10,d10
k2,c20,d20
k3,c30,d30


In [15]:
result = pd.merge(left, right, left_index = True, right_index = True)
result

,a,b,c,d
k0,a10,b10,c10,d10
k2,a12,b12,c20,d20


There is a convenient method from DataFrame -- __join__ that can do the similar job.

In [16]:
result = left.join(right, how = 'inner')
result

,a,b,c,d
k0,a10,b10,c10,d10
k2,a12,b12,c20,d20


The index of the DataFrame can be aligned on the column in another DataFrame while joining two DataFrame:

In [17]:
left = pd.DataFrame({'a' : ['a0', 'a1', 'a2', 'a3'],
                     'b' : ['b0', 'b1', 'b2', 'b3'],
                     'key': ['k0', 'k1', 'k0', 'k1']})
left

,a,b,key
0,a0,b0,k0
1,a1,b1,k1
2,a2,b2,k0
3,a3,b3,k1


In [18]:
right = pd.DataFrame ({'c' : ['c0', 'c1'],
                       'd' : ['d0', 'd1']},
                       index = ['k0', 'k1'])
right 

,c,d
k0,c0,d0
k1,c1,d1


In [19]:
result = pd.merge(left, right, left_on = 'key', right_index = True, how = 'left', sort = False)
result

,a,b,key,c,d
0,a0,b0,k0,c0,d0
1,a1,b1,k1,c1,d1
2,a2,b2,k0,c0,d0
3,a3,b3,k1,c1,d1


In [20]:
left.join(right, on = 'key')

,a,b,key,c,d
0,a0,b0,k0,c0,d0
1,a1,b1,k1,c1,d1
2,a2,b2,k0,c0,d0
3,a3,b3,k1,c1,d1


To join on multiple keys, the passed DataFrame must have a MultiIndex:

In [21]:
left = pd.DataFrame({'A' : ['A0', 'A1', 'A2', 'A3'],
                     'B' : ['B0', 'B1', 'B2', 'B3'], 
                     'key1' : ['K0', 'K0', 'K1', 'K2'],
                     'key2' : ['K0', 'K1', 'K0', 'K1']})

index = pd.MultiIndex.from_tuples([('K0', 'K0'), ('K1', 'K0'),
                                   ('K2', 'K0'), ('K2', 'K1')])
left

,A,B,key1,key2
0,A0,B0,K0,K0
1,A1,B1,K0,K1
2,A2,B2,K1,K0
3,A3,B3,K2,K1


In [22]:
right = pd.DataFrame({'C' : ['C0', 'C1', 'C2', 'C3'],
                      'D' : ['D0', 'D1', 'D2', 'D3']},
                      index = index)
right

C   D
K0 K0  C0  D0
K1 K0  C1  D1
K2 K0  C2  D2
   K1  C3  D3

In [23]:
result = left.join (right, on = ['key1', 'key2'])
result

,A,B,key1,key2,C,D
0,A0,B0,K0,K0,C0,D0
1,A1,B1,K0,K1,NaN,NaN
2,A2,B2,K1,K0,C1,D1
3,A3,B3,K2,K1,C3,D3


In [24]:
result = pd.merge(left, right, left_on = ['key1', 'key2'], right_index = True, how = 'left')
result

,A,B,key1,key2,C,D
0,A0,B0,K0,K0,C0,D0
1,A1,B1,K0,K1,NaN,NaN
2,A2,B2,K1,K0,C1,D1
3,A3,B3,K2,K1,C3,D3


One can join multiple DataFrame at the same time.

In [25]:
left = pd.DataFrame({'k' : ['K0', 'K1', 'K2'], 'v': [1, 2, 3]})
left = left.set_index('k')
left

,v
k,
K0,1
K1,2
K2,3


In [26]:
right = pd.DataFrame({'k' : ['K0', 'K0', 'K3'], 'v' : [4, 5, 6]})
right = right.set_index('k')
right

,v
k,
K0,4
K0,5
K3,6


In [27]:
right2 = pd.DataFrame({'v' : [7, 8, 9]}, index = ['K1', 'K1', 'K2'])
right2

,v
K1,7
K1,8
K2,9


In [28]:
result = left.join([right, right2])
result

,v_x,v_y,v
K0,1.0,4.0,NaN
K0,1.0,5.0,NaN
K1,2.0,NaN,7.0
K1,2.0,NaN,8.0
K2,3.0,NaN,9.0
K3,NaN,6.0,NaN


## Concatenating along the axis
Normally there are several issues the concatenating needs to address:
* If the objects are indexed differently on the other axis, should the collection of axes be unioned or intersected?
* Do the groups need to be identifiable in the resulting object?
* Does the concatenation axis matter at all?

In [29]:
df1 = pd.DataFrame({'a' :['a0', 'a1', 'a2'],
                    'b' :['b0', 'b1', 'b2'],
                    'c' :['c0', 'c1', 'c2']})
df1

,a,b,c
0,a0,b0,c0
1,a1,b1,c1
2,a2,b2,c2


In [30]:
df2 = pd.DataFrame({'a' : ['a3', 'a4', 'a5'],
                    'b' : ['b3', 'b4', 'b5'],
                    'c' : ['c3', 'c4', 'c5']})
df2

,a,b,c
0,a3,b3,c3
1,a4,b4,c4
2,a5,b5,c5


In [31]:
df3 = pd.DataFrame({'a' : ['a6', 'a7', 'a8'],
                    'b' : ['b6', 'b7', 'b8'],
                    'c' : ['c6', 'c7', 'c8']})
df3

,a,b,c
0,a6,b6,c6
1,a7,b7,c7
2,a8,b8,c8


In [32]:
pd.concat([df1, df2, df3])

,a,b,c
0,a0,b0,c0
1,a1,b1,c1
2,a2,b2,c2
0,a3,b3,c3
1,a4,b4,c4
2,a5,b5,c5
0,a6,b6,c6
1,a7,b7,c7
2,a8,b8,c8


In [33]:
pd.concat([df1, df2, df3], axis = 1)

,a,b,c,a,b,c,a,b,c
0,a0,b0,c0,a3,b3,c3,a6,b6,c6
1,a1,b1,c1,a4,b4,c4,a7,b7,c7
2,a2,b2,c2,a5,b5,c5,a8,b8,c8


By default, the concatenating is joining in a _outer_ way, while you can choose an _inner_ way by speicfying __join="inner"__.

The concatenated pieces are not identifiable in the result by default. In order to identify the original pieces, one can specific keys with each of the pieces of the chopped up DataFrame. 

In [34]:
result = pd.concat([df1, df2, df3], keys = ['x', 'y', 'z'])
result

a   b   c
x 0  a0  b0  c0
  1  a1  b1  c1
  2  a2  b2  c2
y 0  a3  b3  c3
  1  a4  b4  c4
  2  a5  b5  c5
z 0  a6  b6  c6
  1  a7  b7  c7
  2  a8  b8  c8

In [35]:
result.ix['z']

,a,b,c
0,a6,b6,c6
1,a7,b7,c7
2,a8,b8,c8


There is some situation that the row index is not meaningful in the context of the anaylsis.

In [36]:
import numpy as np
df1a  = pd.DataFrame(np.random.randn(3, 4), columns = ['a', 'b', 'c', 'd'])
df1a

,a,b,c,d
0,0.032899,2.040317,0.441418,1.714860
1,-0.741990,1.337179,-0.113107,0.207354
2,0.955000,0.397571,1.673367,0.774151


In [37]:
df2a = pd.DataFrame(np.random.randn(2, 3), columns = ['b', 'd', 'a'])
df2a

,b,d,a
0,0.450574,-1.634921,0.906603
1,1.041582,-0.508622,-0.265118


In [39]:
pd.concat([df1a, df2a], ignore_index = True)

,a,b,c,d
0,0.032899,2.040317,0.441418,1.714860
1,-0.741990,1.337179,-0.113107,0.207354
2,0.955000,0.397571,1.673367,0.774151
3,0.906603,0.450574,NaN,-1.634921
4,-0.265118,1.041582,NaN,-0.508622


There is an instance method on Series and DataFrame called __append__ that can concatenating.

In [40]:
result = df1.append(df2)
result

,a,b,c
0,a0,b0,c0
1,a1,b1,c1
2,a2,b2,c2
0,a3,b3,c3
1,a4,b4,c4
2,a5,b5,c5


## Combing data with overlap
One may have two datasets whose indexes overlap in full or part, thus one needs to choose one of them in the combinations.

In [41]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index = ['f', 'e', 'd', 'c', 'b', 'a'])
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [42]:
b = pd.Series(np.arange(len(a), dtype = np.float64),
              index = ['f', 'e', 'd', 'c', 'b', 'a'])
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [43]:
b.combine_first(a)

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [44]:
df1 = pd.DataFrame({'a' : [1., np.nan, 5., np.nan],
                    'b' : [np.nan, 2., np.nan, 6.],
                    'c' : range(2, 18, 4)})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [45]:
df2 = pd.DataFrame({'a' : [5., 4., np.nan, 3., 7.],
                    'b' : [np.nan, 3., 4., 6., 8.]})
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [46]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN
